In [1]:
import sf_quant as sf
import polars as pl
import datetime as dt
import numpy as np
import scipy as sp



/home/andre/QuantBattle2025/orthogonal-alphas/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-26 21:29:22,171	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
start = dt.date(1996, 1, 1)
end = dt.date(2024, 12, 31)

columns = [
    'date',
    'barrid',
    'price',
    'return',
    'specific_return',
]

df = pl.read_parquet("russell_3000_daily.parquet")

df.write_parquet("russell_3000_daily.parquet")

FileNotFoundError: No such file or directory (os error 2): russell_3000_daily.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [ ]:
#Helper program to find stocks that have returns over the whole range of dates. I need to simulate the signals somehow.

startDate = dt.date(2013, 1, 1) 
endDate = dt.date(2018, 1, 1)

unique_barrids = df.filter(
    pl.col("date").is_between(startDate, endDate)
).select(pl.col("barrid")).unique()

barrid = unique_barrids.sample(1)[0, 0]
print(barrid)

print(df.filter(
    pl.col("date").is_between(startDate, endDate),
    (pl.col("barrid") == barrid),
).select(
    pl.col("date").alias("date"),
    pl.col("return").alias("return")
))


USAR6S1
shape: (743, 2)
┌────────────┬─────────┐
│ date       ┆ return  │
│ ---        ┆ ---     │
│ date       ┆ f64     │
╞════════════╪═════════╡
│ 2013-01-02 ┆ 3.0365  │
│ 2013-01-03 ┆ 2.7073  │
│ 2013-01-04 ┆ 1.7298  │
│ 2013-01-07 ┆ -0.2429 │
│ 2013-01-08 ┆ -0.4464 │
│ …          ┆ …       │
│ 2015-12-07 ┆ -0.0266 │
│ 2015-12-08 ┆ 0.0266  │
│ 2015-12-09 ┆ 0.0     │
│ 2015-12-10 ┆ 0.0     │
│ 2015-12-11 ┆ 0.0     │
└────────────┴─────────┘


In [ ]:
barrid1 = "USAQ392"
barrid2 = "USAZ6Q1"
barrid3 = "USAROU1"


training_data = df.filter(
    pl.col("date").is_between(startDate, endDate),
    (pl.col("barrid") == barrid1) | (pl.col("barrid") == barrid2) | (pl.col("barrid") == barrid3)
).pivot("barrid", index="date", values="return").select(
    pl.col("date"),
    (pl.col(barrid1)/100).alias("returns1"),
    (pl.col(barrid2)/100).alias("returns2"),
    (pl.col(barrid3)/100).alias("returns3")
)

print(training_data)

#cum_returns = trainingdata.select(pl.col("date"), np.log(pl.col("return") + 1).cum_sum().alias("cumulative_returns"))

#print(cum_returns.select(pl.col("date"), (np.exp(pl.col("cumulative_returns"))-1).alias("Cumulative_Returns")))


shape: (1_259, 4)
┌────────────┬───────────┬───────────┬───────────┐
│ date       ┆ returns1  ┆ returns2  ┆ returns3  │
│ ---        ┆ ---       ┆ ---       ┆ ---       │
│ date       ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪═══════════╪═══════════╪═══════════╡
│ 2013-01-02 ┆ 0.005376  ┆ 0.022719  ┆ 0.014524  │
│ 2013-01-03 ┆ 0.032086  ┆ -0.008599 ┆ -0.003937 │
│ 2013-01-04 ┆ 0.036269  ┆ 0.010481  ┆ -0.017248 │
│ 2013-01-07 ┆ -0.0002   ┆ 0.010372  ┆ 0.001097  │
│ 2013-01-08 ┆ -0.002801 ┆ 0.007788  ┆ 0.011687  │
│ …          ┆ …         ┆ …         ┆ …         │
│ 2017-12-22 ┆ -0.033333 ┆ 0.000254  ┆ 0.007785  │
│ 2017-12-26 ┆ 0.0       ┆ 0.002284  ┆ 0.002107  │
│ 2017-12-27 ┆ 0.0       ┆ -0.000506 ┆ -0.001402 │
│ 2017-12-28 ┆ -0.002463 ┆ -0.003799 ┆ 0.009123  │
│ 2017-12-29 ┆ -0.001235 ┆ -0.005594 ┆ 0.004172  │
└────────────┴───────────┴───────────┴───────────┘


In [ ]:

def cov_error(r, b): #ratio for geometric weighting.
    error = 0.01 #approximately the contribution of the last day included in the average.
    
    num_r = int(np.floor_divide(np.log(error), np.log(r))) #num_rber of days computed in the average.
    weights_r = [(r**i) for i in range(1, num_r + 1)] #weights_r for the rolling sum. Note they are automatially normalized.
    
    num_b = int(np.floor_divide(np.log(error), np.log(b))) #num_rber of days computed in the average.
    weights_b = [(b**i) for i in range(1, num_b + 1)] #weights_r for the rolling sum. Note they are automatially normalized.

    exprs_cov_construction = [pl.col("date").alias("date")] #Expressions for constructing columns representing covariance matrix coefficients.
    for i in range(1, 4):
        for j in range(1, 4):
            exprs_cov_construction.append(
                ((pl.col(f"returns{i}") - pl.col(f"mu{i}")) *
                (pl.col(f"returns{j}") - pl.col(f"mu{j}"))).alias(f"cov{i}{j}")
            )
            
    exprs_cov_est = [pl.col("date").alias("date")] #Expressions for constructing estimated covariance matrix coefficients.
    for i in range(1, 4):
        for j in range(1, 4):
            exprs_cov_est.append(
                pl.col(f"cov{i}{j}").rolling_mean(num_r, weights_r).fill_null(strategy="backward").alias(f"est_cov{i}{j}")
            )

    cov_data = training_data.with_columns(
        pl.col("returns1").rolling_mean(num_b, weights_b).fill_null(strategy="backward").alias("mu1"),
        pl.col("returns2").rolling_mean(num_b, weights_b).fill_null(strategy="backward").alias("mu2"),
        pl.col("returns3").rolling_mean(num_b, weights_b).fill_null(strategy="backward").alias("mu3")
    ).with_columns( #Need the mu's to compute the covariance matrices.
        exprs_cov_construction
    ).with_columns( #Need the covariance matrix coefficients to compute the estimated future covariance matrix.
        exprs_cov_est
    ).with_columns(
        np.sqrt(sum(np.square(pl.col(f"cov{i}{j}").shift(-1) - pl.col(f"est_cov{i}{j}")) for i in range(1, 4) for j in range(1, 4))).alias("cov_error"), #Error in the estimated covariance matrix 
        np.sqrt(sum(np.square(pl.col(f"cov{i}{i}").shift(-1) - pl.col(f"est_cov{i}{i}")) for i in range(1, 3))).alias("cov_error_null"), #This is to test that we actually get an improved error estimate when we consider covariances (not just variances)
        np.sqrt(sum(np.square(pl.col(f"cov{i}{j}")) for i in range(1, 4) for j in range(1, 4))).alias("cov_mag")
    ).with_columns(
        np.divide(pl.col("cov_error"), pl.col("cov_mag")).alias("cov_rel_error")
    ).filter(
        pl.col("date").is_between(startDate + dt.timedelta(num_r + num_b), endDate)
        #rolling_mean introduces a null in a row when there are too many weights_r for the num_rber of available elements. 
        #this is just filtering out backfilled nulls from two rolling_mean steps.
    )
    #pl.col(f"est_cov{i}{j}") is the estimated future covariance matrix at time step t. We need to compare it to the actual covariance matrix at time t+1.

    #print(cov_data)

    #We now compute the element-wise squared error of the estimated covariance matrix for each t.
    cov_error = cov_data.select(pl.mean("cov_error"))[0, 0]
    cov_mag = cov_data.select(pl.mean("cov_mag"))[0, 0]
    cov_error_null = cov_data.select(pl.mean("cov_error_null"))[0, 0]

    #cov_rel_error = cov_data.select(pl.mean("cov_rel_error"))[0, 0]  #I think volatility makes this a bad measure

    return cov_error, cov_mag, cov_error_null, [r, b], (num_r + num_b)



In [ ]:
for i in range(1, 10):
    print(f"i = {i}, Cov Error: {cov_error(i/10.0, 0.9)[0]}, Cov Mag: {cov_error(i/10.0, 0.9)[1]}")

i = 1, Cov Error: 0.0017122572149002948, Cov Mag: 0.0011935513072305309
i = 2, Cov Error: 0.0016705854516756802, Cov Mag: 0.0011935513072305309
i = 3, Cov Error: 0.0016213574310335777, Cov Mag: 0.0011915176543595778
i = 4, Cov Error: 0.0015654773420463704, Cov Mag: 0.0011915176543595778
i = 5, Cov Error: 0.0015326943824483094, Cov Mag: 0.0011915176543595778
i = 6, Cov Error: 0.001474680848593366, Cov Mag: 0.0011884507808995657
i = 7, Cov Error: 0.001451157849326301, Cov Mag: 0.0011889302898977175
i = 8, Cov Error: 0.001410961202641907, Cov Mag: 0.0011864094635682322
i = 9, Cov Error: 0.0013218169479247222, Cov Mag: 0.0011900596504356609


In [ ]:
U = 950
L = 900

M = 1000.0
Len = (U-L)
Cov_Errors = [[cov_error(i/M, j/M) for i in range(L, U)] for j in range(L, U)]





In [ ]:
training_days = (startDate - endDate).days

Sample_Size = [[Cov_Errors[i-L][j-L][4] for i in range(L, U)] for j in range(L, U)]

Abs_Errors = [[Cov_Errors[i-L][j-L][0]*(1+np.divide(1, Sample_Size[i-L][j-L]-1)) for i in range(L, U)] for j in range(L, U)]
Cov_Mag = [[Cov_Errors[i-L][j-L][1] for i in range(L, U)] for j in range(L, U)]

Rel_Errors = [[np.divide(Abs_Errors[i-L][j-L], Cov_Mag[i-L][j-L]) for i in range(L, U)] for j in range(L, U)]

min = np.argmin(Rel_Errors)

print(Rel_Errors[min//Len][min%Len])
print(Cov_Errors[min//Len][min%Len])



1.1083529920168278
(0.0012936687827024267, 0.0011785312830885496, 0.0008603427786633642, [0.9164, 0.9153], 104)


0.958157121980979
(8.415090972179432e-06, 8.86868308873483e-06, [0.9152, 0.9153], 103)